In [5]:
from sklearn.pipeline import make_pipeline

In [6]:
import pandas as pd

In [7]:
from sklearn.pipeline import Pipeline

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
import numpy as np

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
Data = pd.read_csv("Training_set.csv")

In [13]:
Data_Labels=Data["Heart Attack Risk"]

In [14]:
Data.dropna(subset=['Blood Pressure'], inplace=True)
Data[['Systolic Blood Pressure', 'Diastolic Blood Pressure']] = Data['Blood Pressure'].str.split('/', expand=True).astype(float)
Data.drop("Blood Pressure",axis=1,inplace=True)

Splitting the data into numerical and categorical data

In [16]:
Data_nums = Data.select_dtypes(include=[np.number])

In [17]:
Data_cat = Data.select_dtypes(include=[object])

Creating a numerical pipeline

In [25]:
Data_cat = Data_cat.drop("Patient ID",axis=1)

In [19]:
num_pipeline = Pipeline([
("impute", SimpleImputer(strategy="median")),
("standardize", StandardScaler()),
])

In [20]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

In [21]:
num_prepared = num_pipeline.fit_transform(Data_nums)

In [22]:
num_prepared = pd.DataFrame(
 num_prepared,
 columns=num_pipeline.get_feature_names_out(),
 index=Data_nums.index
)

In [23]:
print(num_prepared[:2].round(2))

    Age  Cholesterol  Heart Rate  Diabetes  Family History  Smoking  Obesity  \
0 -1.40         0.29       -0.58     -1.38           -0.99     0.34     0.99   
1 -0.27         1.24       -0.44     -1.38           -0.99     0.34    -1.01   

   Alcohol Consumption  Exercise Hours Per Week  Previous Heart Problems  ...  \
0                 0.83                    -0.85                     -1.0  ...   
1                 0.83                    -1.59                     -1.0  ...   

   Stress Level  Sedentary Hours Per Day  Income   BMI  Triglycerides  \
0         -0.52                    -0.79    1.74 -0.83           1.24   
1         -0.17                    -1.01    0.11  1.06          -1.51   

   Physical Activity Days Per Week  Sleep Hours Per Day  Heart Attack Risk  \
0                             0.66                 0.99              -0.74   
1                            -1.10                 0.99              -0.74   

   Systolic Blood Pressure  Diastolic Blood Pressure  
0    

Creating a categorical pipeline 

In [26]:
cat_pipeline = Pipeline([
("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [27]:
cat_pipeline = make_pipeline(OneHotEncoder())

In [28]:
cat_prepared = cat_pipeline.fit_transform(Data_cat)

In [29]:
cat_prepared= cat_prepared.toarray()

In [30]:
print(cat_prepared[:2])

[[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [31]:
print(cat_prepared.shape)

(6984, 38)


In [32]:
z = np.concatenate((num_prepared, cat_prepared), axis=1)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [34]:
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(z, Data_Labels)

ValueError: Number of labels=7001 does not match number of samples=6984